# Live Model Server Testing

Test model server via HTTP calls 

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config spec.image = "mlrun/mlrun"

%nuclio: setting spec.image to 'mlrun/mlrun'


In [3]:
import os
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime
from mlrun.datastore import DataItem
from mlrun.artifacts import ChartArtifact
import mlrun

def model_server_tester(context,
                        table: DataItem,
                        addr: str, 
                        label_column: str = "label",
                        model: str = '',
                        match_err: bool = False,
                        rows: int = 20):
    """ Test a model server 
    
    :param table:         csv/parquet table with test data
    :param addr:          function address/url
    :param label_column:  name of the label column in table
    :param model:         tested model name 
    :param match_err:     raise error on validation (require proper test set)
    :param rows:          number of rows to use from test set
    """
        
    table = table.as_df()

    y_list = table.pop(label_column).values.tolist()
    context.logger.info(f'testing with dataset against {addr}, model: {model}')
    if rows and rows < table.shape[0]:
        table = table.sample(rows)
    
    count = err_count = match = 0
    times = []
    for x, y in zip(table.values, y_list):
        count += 1
        event_data = json.dumps({"inputs":[x.tolist()]})
        had_err = False
        try:
            start = datetime.now()
            resp = requests.put(f'{addr}/v2/models/{model}/infer', json=event_data)
            if not resp.ok:
                context.logger.error(f'bad function resp!!\n{resp.text}')
                err_count += 1
                continue
            times.append((datetime.now()-start).microseconds)
                
        except OSError as err:
            context.logger.error(f'error in request, data:{event_data}, error: {err}')
            err_count += 1
            continue
        
        resp_data = resp.json()
        print(resp_data)
        y_resp = resp_data['outputs'][0]
        if y == y_resp:
            match += 1
        
    context.log_result('total_tests', count)
    context.log_result('errors', err_count)
    context.log_result('match', match)
    if count - err_count > 0:
        times_arr = np.array(times)
        context.log_result('avg_latency', int(np.mean(times_arr)))
        context.log_result('min_latency', int(np.amin(times_arr)))
        context.log_result('max_latency', int(np.amax(times_arr)))
        
        chart = ChartArtifact('latency', header=['Test', 'Latency (microsec)'])
        for i in range(len(times)):
            chart.add_row([i+1, int(times[i])])
        context.log_artifact(chart)

    context.logger.info(f'run {count} tests, {err_count} errors and {match} match expected value')
    
    if err_count:
        raise ValueError(f'failed on {err_count} tests of {count}')
    
    if match_err and match != count:
        raise ValueError(f'only {match} results match out of {count}')

> 2020-10-19 14:06:55,501 [warning] Failed resolving version info. Ignoring and using defaults


In [4]:
# nuclio: end-code
# marks the end of a code section

### Deploy model server for testing

In [6]:
project_name = 'sk-project'
DATA_PATH = 'https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv'
MODEL_PATH = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'

artifact_path = mlrun.set_environment(api_path = 'http://mlrun-api:8080',
                                      artifact_path = os.path.abspath('./'))

fn = mlrun.import_function('hub://v2_model_server')
fn.add_model('mymodel', model_path=MODEL_PATH)
address = fn.deploy()

> 2020-10-19 14:07:59,172 [warning] warning!, server (0.5.2) and client (unstable) ver dont match
> 2020-10-19 14:07:59,483 [info] deploy started
[nuclio] 2020-10-19 14:08:08,696 (info) Build complete
[nuclio] 2020-10-19 14:08:16,784 (info) Function deploy complete
[nuclio] 2020-10-19 14:08:16,791 done creating default-v2-model-server, function address: 192.168.224.209:30465
> 2020-10-19 14:08:16,798 [warning] warning!, server (0.5.2) and client (unstable) ver dont match


### Run model server tester locally

In [7]:
gen = mlrun.run_local(name='model_server_tester', handler=model_server_tester, 
                      params={'addr': address, 'model': 'mymodel'},
                      inputs={'table': DATA_PATH},
                      project=project_name, 
                      artifact_path=os.path.join(artifact_path, 'data')) 

> 2020-10-19 14:08:59,455 [warning] warning!, server (0.5.2) and client (unstable) ver dont match
> 2020-10-19 14:08:59,456 [info] starting run model_server_tester uid=67e35351a29e4c7f96be348639675926  -> http://mlrun-api:8080
> 2020-10-19 14:08:59,487 [warning] warning!, server (0.5.2) and client (unstable) ver dont match
> 2020-10-19 14:09:00,339 [info] testing with dataset against http://192.168.224.209:30465, model: mymodel
{'id': '4a399acc-c5e4-48e9-a4a9-935b3cf375ec', 'model_name': 'mymodel', 'outputs': [0]}
{'id': '0fd0161a-9489-451d-bd85-197ab55b86f6', 'model_name': 'mymodel', 'outputs': [2]}
{'id': '1535cf56-8d2a-4e10-8ae6-f8f155e47285', 'model_name': 'mymodel', 'outputs': [1]}
{'id': '684baf71-4dd4-4081-989e-e1a658abfa6c', 'model_name': 'mymodel', 'outputs': [1]}
{'id': 'b7ab113e-2c10-4f6c-9cc4-84f9e2a72b8b', 'model_name': 'mymodel', 'outputs': [0]}
{'id': 'd48d231d-e53a-40d0-812a-32f4b5db3286', 'model_name': 'mymodel', 'outputs': [1]}
{'id': '1481d56c-5e0d-488e-84b1-680f4877

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...39675926,0,Oct 19 14:08:59,completed,model_server_tester,v3io_user=idanbkind=handlerowner=idanbhost=idan-jupyter-5f8568c8b5-4lw4b,table,addr=http://192.168.224.209:30465model=mymodel,total_tests=20errors=0match=4avg_latency=36683min_latency=35941max_latency=39620,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run 67e35351a29e4c7f96be348639675926 --project sk-project , !mlrun logs 67e35351a29e4c7f96be348639675926 --project sk-project
> 2020-10-19 14:09:01,192 [info] run executed, status=completed


### Save

In [8]:
test_func = mlrun.code_to_function(name='v2_model_tester', 
                                   kind='job', 
                                   handler="model_server_tester",
                                   description="test v2 model servers",
                                   categories=["ml", "test"],
                                   labels={"author": "yaronh"},
                                   code_output='.')

test_func.export('function.yaml')

> 2020-10-19 14:10:06,727 [info] function spec saved to path: function.yaml


### Run remotely

In [9]:
test_func.run(mlrun.NewTask(name='model_server_tester', 
                            handler=model_server_tester, 
                            params={'addr': address, 'model': 'mymodel'},
                            inputs={'table': DATA_PATH},
                            project=project_name, 
                            artifact_path=os.path.join(artifact_path, 'data')))

> 2020-10-19 14:10:28,726 [warning] warning!, server (0.5.2) and client (unstable) ver dont match
> 2020-10-19 14:10:28,727 [info] starting run model_server_tester uid=9e110389ce714e1195874ee3f6c35bd6  -> http://mlrun-api:8080
> 2020-10-19 14:10:28,846 [info] Job is running in the background, pod: model-server-tester-dnjqm


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...f6c35bd6,0,Oct 19 14:10:28,running,model_server_tester,v3io_user=idanbkind=jobowner=idanb,table,addr=http://192.168.224.209:30465model=mymodel,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 9e110389ce714e1195874ee3f6c35bd6 --project sk-project , !mlrun logs 9e110389ce714e1195874ee3f6c35bd6 --project sk-project
> 2020-10-19 14:10:28,897 [info] run executed, status=running
